你想实现一个状态机，或者在不同状态下执行操作的对象，但是又不希望在代码中出现太多的条件判断语句

In [1]:
# 普通方案，夹杂着大量的判断语句，效率低下
class Connection:
    def __init__(self):
        self.state = 'CLOSED'
    
    def read(self):
        if self.state != 'OPEN':
            raise RuntimeError('Not open')
        print ('reading')
    
    def write(self):
        if self.state != 'OPEN':
            raise RuntimeError('Not open')
        print('writing')

    
    def open(self):
        if self.state == 'OPEN':
            raise RuntimeError('Already open')
        self.state = 'OPEN'
        
    
    def close(self):
        if self.state == 'CLOSED':
            raise RuntimeError('already closed')
        self.state = 'CLOSED'

这样写有两个缺点：   
1、代码复杂，好多条件判断   
2、执行效率低下，因为一些常见操作比如read()/write()每次执行前都需要执行检查

一个更好的办法是为每个状态定义一个对象：

In [2]:
class Connection1:
    """新方案——对每个状态定义一个类"""

    def __init__(self):
        self.new_state(ClosedConnectionState)

    def new_state(self, newstate):
        self._state = newstate
        # Delegate to the state class

    def read(self):
        return self._state.read(self)

    def write(self, data):
        return self._state.write(self, data)

    def open(self):
        return self._state.open(self)

    def close(self):
        return self._state.close(self)


# Connection state base class
class ConnectionState:
    @staticmethod
    def read(conn):
        raise NotImplementedError()

    @staticmethod
    def write(conn, data):
        raise NotImplementedError()

    @staticmethod
    def open(conn):
        raise NotImplementedError()

    @staticmethod
    def close(conn):
        raise NotImplementedError()


# Implementation of different states
class ClosedConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        raise RuntimeError('Not open')

    @staticmethod
    def write(conn, data):
        raise RuntimeError('Not open')

    @staticmethod
    def open(conn):
        conn.new_state(OpenConnectionState)

    @staticmethod
    def close(conn):
        raise RuntimeError('Already closed')


class OpenConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        print('reading')

    @staticmethod
    def write(conn, data):
        print('writing')

    @staticmethod
    def open(conn):
        raise RuntimeError('Already open')

    @staticmethod
    def close(conn):
        conn.new_state(ClosedConnectionState)